In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from joblib import Parallel, delayed
from collections import Counter
import warnings


In [2]:
# ✅ 데이터 로드
train = pd.read_csv('processed_train.csv')
test = pd.read_csv('test.csv')

# ✅ 가중치 데이터 로드
weight_data = pd.read_csv('og_weighted_hong.csv', encoding='utf-8')
weight_dict = weight_data.set_index("데이터 항목").to_dict()

# ✅ 가중치 적용 (병렬 처리)
def apply_feature_weights(X, weight_dict):
    X_weighted = X.copy()
    
    def process_category(category, weights):
        common_cols = list(set(weights.keys()).intersection(X.columns))
        if common_cols:
            X_weighted[common_cols] *= pd.DataFrame(weights, index=X.index, columns=common_cols)

    Parallel(n_jobs=-1)(delayed(process_category)(category, weights) for category, weights in weight_dict.items())
    return X_weighted

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

X_weighted = apply_feature_weights(X, weight_dict)
X_test_weighted = apply_feature_weights(test, weight_dict)

# ✅ NumPy 벡터 연산으로 빠른 언더샘플링 적용
counter = Counter(y)
min_class = min(counter, key=counter.get)
num_samples = counter[min_class]

majority_idx = np.where(y == 0)[0]
sampled_majority_idx = np.random.choice(majority_idx, size=num_samples, replace=False)
minority_idx = np.where(y == 1)[0]
undersampled_idx = np.concatenate([sampled_majority_idx, minority_idx])

X_resampled = X_weighted.iloc[undersampled_idx]
y_resampled = y.iloc[undersampled_idx]


In [3]:
# ✅ 범주형 변수 변환 최적화
category_columns = [
    "시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형",
    "배아 생성 주요 이유", "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이"
]

for col in category_columns:
    X_resampled[col] = X_resampled[col].astype("category").cat.codes.astype("int8")
    X_test_weighted[col] = X_test_weighted[col].astype("category").cat.codes.astype("int8")

# ✅ ID 컬럼 삭제
X_resampled = X_resampled.drop(columns=['ID'], errors='ignore')
X_test_weighted = X_test_weighted.drop(columns=['ID'], errors='ignore')

# ✅ Object 타입 컬럼 변환 (XGBoost 오류 방지)
for col in X_resampled.select_dtypes(include=['object']).columns:
    X_resampled[col] = X_resampled[col].astype(str).astype("category")

for col in X_test_weighted.select_dtypes(include=['object']).columns:
    X_test_weighted[col] = X_test_weighted[col].astype(str).astype("category")



C:\Users\tjddl\AppData\Local\Temp\ipykernel_32512\42424477.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_resampled[col] = X_resampled[col].astype("category").cat.codes.astype("int8")
C:\Users\tjddl\AppData\Local\Temp\ipykernel_32512\42424477.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_resampled[col] = X_resampled[col].astype("category").cat.codes.astype("int8")
C:\Users\tjddl\AppData\Local\Temp\ipykernel_32512\42424477.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [4]:
X_train, X_val, y_train, y_val = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import gc
import torch

# ✅ 메모리 정리 (GPU 충돌 방지)
gc.collect()
torch.cuda.empty_cache()

# ✅ StackingClassifier (CatBoost는 CPU 사용)
stack_clf = StackingClassifier(
    estimators=[
        ('xgb', XGBClassifier(tree_method='gpu_hist', enable_categorical=True, gpu_id=0, max_bin=256, random_state=42)),
        ('lgbm', LGBMClassifier(device='gpu', gpu_platform_id=0, gpu_device_id=0, max_bin=255, random_state=42)),
        ('cat', CatBoostClassifier(task_type='CPU', depth=6, iterations=300, learning_rate=0.05, verbose=0))
    ],
    final_estimator=Pipeline([
        ('scaler', RobustScaler()),
        ('lr', LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear'))
    ]),
    cv=3,
    n_jobs=1  # ✅ 병렬 실행 X (GPU 충돌 방지)
)

# ✅ RandomizedSearchCV 설정
param_dist = {
    'xgb__n_estimators': [300, 400, 500],
    'xgb__max_depth': [4, 5, 6],
    'xgb__learning_rate': [0.025, 0.05, 0.1],
    'lgbm__n_estimators': [300, 400, 500],
    'lgbm__num_leaves': [31, 63],
    'lgbm__min_child_samples': [20, 50],
    'cat__iterations': [100, 300, 500],
    'cat__depth': [4, 5, 6],
    'cat__learning_rate': [0.025, 0.05, 0.1]
}

random_search = RandomizedSearchCV(
    stack_clf, param_distributions=param_dist, 
    n_iter=20,  
    scoring='roc_auc', 
    cv=3,  
    n_jobs=1,  # ✅ CPU & GPU 안정성 증가
    random_state=42, verbose=2
)

# ✅ 모델 학습
random_search.fit(X_train, y_train)

# ✅ 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# ✅ 모델 평가
y_val_pred = random_search.best_estimator_.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f"Validation ROC AUC: {roc_auc:.5f}")

# ✅ 테스트 데이터 예측
pred_proba = random_search.best_estimator_.predict_proba(X_test_weighted)[:, 1]

# ✅ 결과 저장
submission = pd.DataFrame({'ID': [f"TEST_{i:05d}" for i in range(len(test))],'probability': pred_proba})
submission.to_csv('Result_submit.csv', index=False)
print("✅ 제출 파일 생성 완료!")


Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35321, number of negative: 35321
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 625
[LightGBM] [Info] Number of data points in the train set: 70642, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 15 dense feature groups (1.08 MB) transferred to GPU in 0.003005 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\tjddl\AppData\Local\Programs\Python\Python310

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23547, number of negative: 23547
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 607
[LightGBM] [Info] Number of data points in the train set: 47094, number of used features: 61
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 15 dense feature groups (0.72 MB) transferred to GPU in 0.001902 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23547, number of negative: 23548
[LightGBM] [Info] This is the 

...

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:50:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:50:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\tjddl\AppData\Local\Programs\Python\Python310

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35321, number of negative: 35321
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 621
[LightGBM] [Info] Number of data points in the train set: 70642, number of used features: 60
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 15 dense feature groups (1.08 MB) transferred to GPU in 0.002234 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35321, number of negative: 35322
[LightGBM] [Info] This is the 

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:51:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Validation ROC AUC: 0.73652


ValueError: train and valid dataset categorical_feature do not match.